In [1]:
"""
note
2025.03.13
미니 프로젝트 시작.
상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.
2025.03.14
넘파이 없이 구현을 목표로 했으나 특징벡터 4개, 클래스 3개로 계산량이 너무 커서 넘파이를 사용해 구현,
추후 상관계수 계산 후 특징벡터 2개를 이용한 이진, 혹은 3 클래스 분류를 진행하고자함.
"""

'\nnote\n2025.03.13\n미니 프로젝트 시작.\n상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.\n2025.03.14\n넘파이 없이 구현을 목표로 했으나 특징벡터 4개, 클래스 3개로 계산량이 너무 커서 넘파이를 사용해 구현,\n추후 상관계수 계산 후 특징벡터 2개를 이용한 이진, 혹은 3 클래스 분류를 진행하고자함.\n'

In [74]:
from typing import Tuple
import csv
import numpy as np

np.random.seed(0)

In [75]:
f = open("Iris.csv")
rdr = csv.reader(f)

In [76]:
csv_data = [i for i in rdr]
columns = csv_data[0]
data = csv_data[1:]

In [77]:
columns, len(data)

(['Id',
  'SepalLengthCm',
  'SepalWidthCm',
  'PetalLengthCm',
  'PetalWidthCm',
  'Species'],
 150)

In [78]:
set(i[5] for i in data)

{'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}

In [79]:
_target = {'Iris-versicolor':0, 'Iris-setosa':1, 'Iris-virginica':2}    # Sepal: 꽃받침, Petal: 꽃잎

data = np.array([ [id, *i[1:-1], _target[i[-1]] ] for id, i in enumerate(data) ])
columns = np.array(['Id', 'SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'target'])

data, columns, data.shape

(array([['0', '5.1', '3.5', '1.4', '0.2', '1'],
        ['1', '4.9', '3.0', '1.4', '0.2', '1'],
        ['2', '4.7', '3.2', '1.3', '0.2', '1'],
        ['3', '4.6', '3.1', '1.5', '0.2', '1'],
        ['4', '5.0', '3.6', '1.4', '0.2', '1'],
        ['5', '5.4', '3.9', '1.7', '0.4', '1'],
        ['6', '4.6', '3.4', '1.4', '0.3', '1'],
        ['7', '5.0', '3.4', '1.5', '0.2', '1'],
        ['8', '4.4', '2.9', '1.4', '0.2', '1'],
        ['9', '4.9', '3.1', '1.5', '0.1', '1'],
        ['10', '5.4', '3.7', '1.5', '0.2', '1'],
        ['11', '4.8', '3.4', '1.6', '0.2', '1'],
        ['12', '4.8', '3.0', '1.4', '0.1', '1'],
        ['13', '4.3', '3.0', '1.1', '0.1', '1'],
        ['14', '5.8', '4.0', '1.2', '0.2', '1'],
        ['15', '5.7', '4.4', '1.5', '0.4', '1'],
        ['16', '5.4', '3.9', '1.3', '0.4', '1'],
        ['17', '5.1', '3.5', '1.4', '0.3', '1'],
        ['18', '5.7', '3.8', '1.7', '0.3', '1'],
        ['19', '5.1', '3.8', '1.5', '0.3', '1'],
        ['20', '5.4', '3.4', '

In [80]:
"""
2025.03.13
상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.
"""

'\n2025.03.13\n상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.\n'

In [81]:
# np.random.shuffle(data): 원본 변경
# np.random.permutation(data): 원본 유지, 수정본 반환
# 데이터의 클래스 비율을 유지 할 수 있었으나 이후 랜덤포레스트 구현 시 발생 할 차이를 보기 위해 랜덤으로 유지함.

def train_test_split(data, ratio=0.7)-> Tuple[list, list]:
    shuffled_data = np.random.permutation(data)
    index = int(len(data)*ratio)
    return shuffled_data[:index], shuffled_data[index:]

In [82]:
trainX, trainY = train_test_split(data)
trainX.shape, trainY.shape

((105, 6), (45, 6))

In [172]:
trainX[:,0] = np.array([i for i in range(trainX.shape[0])])
trainY[:,0] = np.array([i for i in range(trainY.shape[0])])

In [173]:
"""
결정트리분류 모델 구현
손실함수가 필요 없는 모델이며 지니지수를 이용해 데이터를 분류한다.
분류에는 2~4 특징을 이용한다. 성능 평가는 ACC(정확도)를 사용한다.
연결리스트, 노드를 사용한다.
attribute:
DB = Decision_Boundary(결정경계) - 모델로 구하고자 하는 목표, 데이터를 분류하는 기준.
maxG = maxG가 작아지도록 결정경계를 설정한다. 기존값은 1. 결정경계를 생성하면서 점점 낮아진다.

DH = Decision_History - 트리가 클래스를 분류한 기록
"""

'\n결정트리분류 모델 구현\n손실함수가 필요 없는 모델이며 지니지수를 이용해 데이터를 분류한다.\n분류에는 2~4 특징을 이용한다. 성능 평가는 ACC(정확도)를 사용한다.\n연결리스트, 노드를 사용한다.\nattribute:\nDB = Decision_Boundary(결정경계) - 모델로 구하고자 하는 목표, 데이터를 분류하는 기준.\nmaxG = maxG가 작아지도록 결정경계를 설정한다. 기존값은 1. 결정경계를 생성하면서 점점 낮아진다.\n\nDH = Decision_History - 트리가 클래스를 분류한 기록\n'

In [290]:
class DecisionTreeClassifier:
    def __init__(self, depth = 2):
        self.depth = depth                  # 트리의 깊이 = 분할 횟수

        self.DB = [[] for _ in range(2**depth-1)]

        self.data = None
        self.pMap = None
        

    # 서브트리 루트 노드를 받아 최적의 결정경계 산출
    # 입력으로 루트 노드의 데이터 인덱스(Id)를 받음

    def compute(self, _left, _right, _cnt, _idx):       # Gini 불순도 계산 함수
        _GIl = 1 - np.sum((_left / _idx)**2)
        _GIr = 1 - np.sum((_right/(_cnt -_idx))**2)
        return ((_cnt -_idx) *_GIl + _idx * _GIr) /_cnt
        
    def CART(self, _id, _pos):
        print('---CART___')
        x = self.data[_id]
        _db = None
        _cnt = x.shape[0]
        _maxG = 1 - np.sum((np.bincount(x[:, -1].astype(np.int16))/_cnt)**2)   # 데이터를 나누지 않았을 때 불순도

        for i in range(1, self.data.shape[1]-1):                           # 특징벡터 만큼 반복
            _x = x[np.argsort(x[:, i])].view()                             # 단일 특징을 기준으로 정렬된 배열 _x
            
            for _idx in range(1, _x.shape[0]):
                _left = np.bincount(_x[:_idx, -1].astype(np.int16))
                _right = np.bincount(_x[_idx:, -1].astype(np.int16))
                _G = self.compute(_left, _right, _cnt, _idx)
                if (_G < _maxG):
                    _maxG = _G
                    _db = (i, _idx)
                    #_db = [i, np.mean(_x[_idx-1:_idx, i]), _x[:_idx, 0], _x[_idx:, 0]]

        if _db is None:                                             # 불순도를 더 줄이는 결정경계가 없을 때
            self.pMap[_pos] = np.argmax(x[:, -1].astype(np.int16))
            return
        
        _idxDB = 0
        for i in _pos:
            _idxDB = 2 *_idxDB +1 +i
        f, _idx = _db
        if len(_pos) == self.depth-1:                               # 최고 깊이 도달
            self.DB[_idxDB] = f, np.mean(_x[_idx-1:_idx+1, f].astype(np.float64))
            self.pMap[_pos+(0,)] = np.argmax(np.bincount(_x[:_idx, -1].astype(np.int16)))
            self.pMap[_pos+(1,)] = np.argmax(np.bincount(_x[_idx:, -1].astype(np.int16)))

        else:
            print('f, _idx', f, _idx)
            self.DB[_idxDB] = f, np.mean(_x[_idx-1:_idx+1, f].astype(np.float64))
            self.CART(_x[:_idx, 0].astype(np.int16), _pos+(0,)) # left id
            self.CART(_x[_idx:, 0].astype(np.int16), _pos+(1,)) # right id

    def fit(self, x):
        self.data = x
        self.pMap = np.zeros((2,)*self.depth + np.unique(x[:, -1]).shape)    # 에측 결과 대응 맵
        self.CART(list(range(x.shape[0])), ())

    def predict(self, x):
        self.data = x
        

        pass

In [291]:
dtc = DecisionTreeClassifier(3)
dtc.fit(trainX)

---CART___
f, _idx 1 1
---CART___
---CART___
f, _idx 1 1
---CART___
---CART___


In [292]:
dtc.DB, dtc.pMap

([(1, 4.55), [], (1, 4.6), [], [], [], (1, 5.15)],
 array([[[[0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.]]],
 
 
        [[[0., 0., 0.],
          [0., 0., 0.]],
 
         [[1., 1., 1.],
          [2., 2., 2.]]]]))

In [176]:
ttt = (1,0,1)
[2**m+i for m, i in enumerate(ttt)]

[2, 2, 5]

In [ ]:
tt = {'0':0, '1':1, '2':2}
tt['0']

0

In [70]:
(0,)[1:]

()

In [2]:
[[] for _ in range(2**3-1)]

[[], [], [], [], [], [], []]

In [123]:
#tt = np.repeat([[[1],[None]]], 10, axis=0)
tt = np.arange(30).reshape(-1,2)
tt[0, 1] = 10
tt.shape[0]

15

In [150]:
_index = tt[:,0]==0
_index

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [186]:
list(range(tt.shape[0]))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [149]:
tid = [0,1,2,3]
tt[tid], tt

(array([[ 0, 10],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]]),
 array([[ 0, 10],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17],
        [18, 19],
        [20, 21],
        [22, 23],
        [24, 25],
        [26, 27],
        [28, 29]]))

In [186]:
import numpy as np

# (5, 3) 형태의 NumPy 배열 (예제 데이터)
arr = np.array([
    [1, 2, 0],
    [2, 5, 1],
    [3, 1, 2],
    [4, 7, 0],
    [5, 6, 0],
    [6, 3, 1],
    [7, 2, 2],
    [8, 1, 0],
])

# 정렬 기준이 되는 (n,1) 열 선택 (예: 두 번째 열 기준으로 정렬)
column_index = 1  # 기준 열 인덱스
sorted_indices = np.argsort(arr[:, column_index])  # 기준 열을 정렬한 인덱스

# 정렬된 배열
sorted_arr = arr[sorted_indices]

print(sorted_arr)
arr1 = arr[np.argsort(arr[:, 1])][:, [1, -1]]

[[3 1 2]
 [8 1 0]
 [1 2 0]
 [7 2 2]
 [6 3 1]
 [2 5 1]
 [5 6 0]
 [4 7 0]]


In [187]:
tt = [0, 1, 2]
arr1[tt], arr1

(array([[1, 2],
        [1, 0],
        [2, 0]]),
 array([[1, 2],
        [1, 0],
        [2, 0],
        [2, 2],
        [3, 1],
        [5, 1],
        [6, 0],
        [7, 0]]))

In [24]:
t0 = np.bincount(arr1[:, -1])
for _idx in range(arr1.shape[0]):                             
    t1 = np.bincount(arr1[:_idx, -1], minlength=3) / t0
    t2 = np.bincount(arr1[_idx:, -1], minlength=3) / t0
np.add(t1**2, t2**2)

array([0.625, 1.   , 1.   ])

In [23]:
_idx = 2
np.bincount(arr[:,-1]), arr[:,-1]
arr[:_idx, -1].view(), arr[_idx:, -1].view()

(array([0, 1]), array([2, 0, 0, 1, 2, 0]))

In [32]:
tt0 = np.array([[1, 2, 3],[4, 5, 6]])
np.argmax(tt0)

5

In [247]:
t1 = 1; t2 = 2
int(t1 > t2)

0